In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import warnings
import time
warnings.filterwarnings(action='ignore')

In [5]:
# 드라이버 불러오기
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

# Set up the Chrome driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# URL of the website
url = 'https://www.thecocktailproject.com/search-recipes/'

# Open the website
driver.get(url)

# Wait for the page to load
time.sleep(1)

# Find the birthday input fields and enter the date
month_input = driver.find_element(By.ID, 'edit-month-of-birth')
month_input.send_keys('08')

day_input = driver.find_element(By.ID, 'edit-day-of-birth')
day_input.send_keys('20')

year_input = driver.find_element(By.ID, 'year')
year_input.send_keys('1999')
time.sleep(1)

app_submit = driver.find_element(By.CSS_SELECTOR, 'app-submit')
button_within_submit = app_submit.find_element(By.CSS_SELECTOR, 'button#edit-submit')
button_within_submit.click()
time.sleep(1)

reject_all_button = driver.find_element(By.ID, 'onetrust-reject-all-handler')
reject_all_button.click()
time.sleep(1)

#### 여기까지 access


In [53]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

# Set up the Chrome driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# URL of the website
url = 'https://www.thecocktailproject.com/search-recipes/'

# Open the website
driver.get(url)

# Wait for the page to load
time.sleep(1)

# Find the birthday input fields and enter the date
month_input = driver.find_element(By.ID, 'edit-month-of-birth')
month_input.send_keys('08')

day_input = driver.find_element(By.ID, 'edit-day-of-birth')
day_input.send_keys('20')

year_input = driver.find_element(By.ID, 'year')
year_input.send_keys('1999')
time.sleep(1)

app_submit = driver.find_element(By.CSS_SELECTOR, 'app-submit')
button_within_submit = app_submit.find_element(By.CSS_SELECTOR, 'button#edit-submit')
button_within_submit.click()
time.sleep(1)

reject_all_button = driver.find_element(By.ID, 'onetrust-reject-all-handler')
reject_all_button.click()
time.sleep(1)

#### 여기까지 access

# Find all cocktail elements
cocktail_elements = driver.find_elements(By.CSS_SELECTOR, '.receipe-grid-items')

# Initialize an empty dictionary to store cocktail names and links
cocktails_dict = {}

# Loop through each cocktail element
for cocktail_element in cocktail_elements:
    # Extract name of the cocktail
    name = cocktail_element.find_element(By.CSS_SELECTOR, '.item-desc a').text.strip()
    
    # Extract link of the cocktail
    link = cocktail_element.find_element(By.CSS_SELECTOR, '.item-detail a').get_attribute('href')
    
    # Add the name and link of the cocktail to the dictionary
    cocktails_dict[name] = link

len(cocktails_dict)
# 첫 장 칵테일만 크롤링 완료 - 24개
# 크롤링 잘 되는 거 확인하고 22페이지 for loop 돌리기

24

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

# Set up the Chrome driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# URL of the website
url = 'https://www.thecocktailproject.com/search-recipes/'

# Open the website
driver.get(url)

# Wait for the page to load
time.sleep(1)

# Find the birthday input fields and enter the date
month_input = driver.find_element(By.ID, 'edit-month-of-birth')
month_input.send_keys('08')

day_input = driver.find_element(By.ID, 'edit-day-of-birth')
day_input.send_keys('20')

year_input = driver.find_element(By.ID, 'year')
year_input.send_keys('1999')
time.sleep(1)

app_submit = driver.find_element(By.CSS_SELECTOR, 'app-submit')
button_within_submit = app_submit.find_element(By.CSS_SELECTOR, 'button#edit-submit')
button_within_submit.click()
time.sleep(1)

reject_all_button = driver.find_element(By.ID, 'onetrust-reject-all-handler')
reject_all_button.click()
time.sleep(1)

#### 여기까지 access

# Find all cocktail elements
# Initialize an empty dictionary to store cocktail names and links
cocktails_dict = {}

# Loop through each page from 1 to 22
for i in range(0, 22):
    # Get the URL of the page
    page_url = f"https://www.thecocktailproject.com/search-recipes/?page={i}"
    # Visit the page
    driver.get(page_url)
    
    # Find all cocktail elements
    cocktail_elements = driver.find_elements(By.CSS_SELECTOR, '.receipe-grid-items')

    # Loop through each cocktail element on the current page
    for cocktail_element in cocktail_elements:
        # Extract name of the cocktail
        name = cocktail_element.find_element(By.CSS_SELECTOR, '.item-desc a').text.strip()

        # Extract link of the cocktail
        link = cocktail_element.find_element(By.CSS_SELECTOR, '.item-detail a').get_attribute('href')

        # Add the name and link of the cocktail to the dictionary
        cocktails_dict[name] = link
        
driver.quit()

In [3]:
len(cocktails_dict)

## 440.. 근데 528이어야함 ㅠ
## 한 페이지에 칵테일 24개, 22페이지 = 24 * 22 = 528
## 이거 다 중복임 ㅆㅃ!!!!!!!!!!!!!!!1 것도 모르고 씨팔

440

In [39]:
cocktails_dict['Lazy Day Colada']
## 칵테일 이름 몇개로 검색해도 잘은 나옴

'https://www.thecocktailproject.com/drink-recipes/lazy-day-colada'

In [91]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

# Set up the Chrome driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# URL of the website
url = 'https://www.thecocktailproject.com/search-recipes/'

# Open the website
driver.get(url)

# Wait for the page to load
time.sleep(1)

# Find the birthday input fields and enter the date
month_input = driver.find_element(By.ID, 'edit-month-of-birth')
month_input.send_keys('08')

day_input = driver.find_element(By.ID, 'edit-day-of-birth')
day_input.send_keys('20')

year_input = driver.find_element(By.ID, 'year')
year_input.send_keys('1999')
time.sleep(1)

app_submit = driver.find_element(By.CSS_SELECTOR, 'app-submit')
button_within_submit = app_submit.find_element(By.CSS_SELECTOR, 'button#edit-submit')
button_within_submit.click()
time.sleep(1)

reject_all_button = driver.find_element(By.ID, 'onetrust-reject-all-handler')
reject_all_button.click()
time.sleep(1)

#### 여기까지 access

# Find all cocktail elements
# Initialize an empty dictionary to store cocktail names and links
cocktails_dict = {}
duplicate_count = {}
duplicate = 0
# Loop through each page from 1 to 22
for i in range(0, 22):
    # Get the URL of the page
    page_url = f"https://www.thecocktailproject.com/search-recipes/?page={i}"
    # Visit the page
    driver.get(page_url)
    
    # Find all cocktail elements
    cocktail_elements = driver.find_elements(By.CSS_SELECTOR, '.receipe-grid-items')

    # Loop through each cocktail element on the current page
    for cocktail_element in cocktail_elements:
        # Extract name of the cocktail
        name = cocktail_element.find_element(By.CSS_SELECTOR, '.item-desc a').text.strip()

        # Extract link of the cocktail
        link = cocktail_element.find_element(By.CSS_SELECTOR, '.item-detail a').get_attribute('href')

        if name in cocktails_dict:
            # Increment the occurrence count of the key
            duplicate_count[name] = duplicate_count.get(name, 0) + 1
            duplicate += 1
        else:
            # Add the name and link of the cocktail to the dictionary
            cocktails_dict[name] = link

# Print the count of duplicate keys
print("Duplicate key count: " + str(duplicate))

driver.quit()

len(cocktails_dict)


Duplicate key count: 88


440

In [106]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Set up the Chrome driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# URL of the website containing cocktails
url = 'https://www.thecocktailproject.com/search-recipes/'

# Open the website
driver.get(url)

# Wait for the page to load and for the cookie popup to disappear
wait = WebDriverWait(driver, 10)
wait.until(EC.invisibility_of_element_located((By.ID, 'onetrust-pc-btn-handler')))

# Initialize an empty dictionary to store cocktail names, links, and image links
cocktails_dict = {}

# Loop through each page from 1 to 22
for i in range(0, 22):
    # Get the URL of the page
    page_url = f"https://www.thecocktailproject.com/search-recipes/?page={i}"
    # Visit the page
    driver.get(page_url)
    
    # Wait for the cocktail elements to be present
    cocktail_elements = wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, '.receipe-grid-items')))

    # Loop through each cocktail element on the current page
    for cocktail_element in cocktail_elements:
        # Extract name of the cocktail
        name = cocktail_element.find_element(By.CSS_SELECTOR, '.item-desc a').text.strip()

        # Extract link of the cocktail
        link = cocktail_element.find_element(By.CSS_SELECTOR, '.item-detail a').get_attribute('href')

        # Extract image link of the cocktail
        image_element = cocktail_element.find_element(By.CSS_SELECTOR, '.field--name-field-recipe-image img')
        image_link = image_element.get_attribute('src')
        
        
        # Add the name, link, and image link of the cocktail to the dictionary
        cocktails_dict[name] = {'link': link, 'image_link': image_link}

# Close the browser
driver.quit()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [10]:
import os
import requests

# Loop through each cocktail in the dictionary
for name, image_link in cocktails_dict.items():
    
    # Save image as PNG with cocktail name as file name
    response = requests.get(image_link)
    if response.status_code == 200:
        # Remove special characters from the name
        name = name.replace("®", "")
        name = name.replace("\n","")
        
        # Define the file path
        file_path = f"cocktail_images/{name}.png"
        
        # Check if the file already exists
        if os.path.exists(file_path):
            os.remove(file_path)  # Remove the existing file
        
        # Write the new image to file
        with open(file_path, "wb") as f:
            f.write(response.content)


In [11]:
import os

# Directory containing the cocktail images
directory = "cocktail_images"

# Get the list of files in the directory
files = os.listdir(directory)

# Count the number of files
num_files = len(files)

# Print the number of files
print(f"Number of files in {directory}: {num_files}")


Number of files in cocktail_images: 450


In [13]:
import requests

directory = "images"
if not os.path.exists(directory):
    os.makedirs(directory)
    
# Loop through each cocktail in the dictionary
for name, image_link in cocktails_dict.items():
    
    # Save image as PNG with cocktail name as file name
    response = requests.get(image_link)
    if response.status_code == 200:
        # Remove special characters from the name
        name = name.replace("®", "")
        name = name.replace("\n","")
        
        # Define the file path
        file_path = f"images/{name}.png"
        
        # Write the new image to file
        with open(file_path, "wb") as f:
            f.write(response.content)


In [14]:
import os
directory = "images"
files = os.listdir(directory)
num_files = len(files)

# Print the number of files
print(f"Number of files in {directory}: {num_files}")

Number of files in images: 440


In [5]:
cocktails_dict.items()

dict_items([('Espresso Martini', 'https://www.thecocktailproject.com/Espresso-Martini'), ('French 75', 'https://www.thecocktailproject.com/French-75'), ('Last Word', 'https://www.thecocktailproject.com/Last-Word'), ('Tom Collins', 'https://www.thecocktailproject.com/Tom-Collins'), ('Tokyo Collins', 'https://www.thecocktailproject.com/Tokyo-Collins'), ('Gin Basil Smash', 'https://www.thecocktailproject.com/drink-recipes/gin-basil-smash'), ('Japanese Gin Sour', 'https://www.thecocktailproject.com/Japanese-Gin-Sour'), ('Gimlet', 'https://www.thecocktailproject.com/drink-recipes/gimlet'), ('Kentucky Mule', 'https://www.thecocktailproject.com/Kentucky-Mule'), ('Vesper', 'https://www.thecocktailproject.com/Vesper'), ('Sazerac', 'https://www.thecocktailproject.com/Sazerac'), ("Bee's Knees", 'https://www.thecocktailproject.com/Bees-Knees'), ('Jim Beam® and Ginger Highball', 'https://www.thecocktailproject.com/drink-recipes/jim-beam-ginger-highball'), ('Midori® Pot of Gold', 'https://www.thecoc

In [109]:
cocktails_dict["Espresso Martini"]

{'link': 'https://www.thecocktailproject.com/Espresso-Martini',
 'image_link': 'https://www.thecocktailproject.com/sites/default/files/styles/grid-thumb/public/EspressoMartini.jpg?itok=161L5upy'}

In [111]:
# Ingredients 뽑는 중
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Set up the Chrome driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# URL of the cocktail detail page
url = 'https://www.thecocktailproject.com/drink-recipes/blue-margarita/'

# Open the cocktail detail page
driver.get(url)

# Wait for the ingredients to load
wait = WebDriverWait(driver, 10)
ingredients_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.ingredients-block')))

# Extract the ingredients
ingredients_html = ingredients_element.get_attribute('innerHTML')
ingredients_soup = BeautifulSoup(ingredients_html, 'html.parser')
ingredients_text = ingredients_soup.get_text(strip=True)

# Split the ingredients into separate lines
ingredients_list = [line.strip() for line in ingredients_text.split('\n') if line.strip()]

print("Ingredients:")
for ingredient in ingredients_list:
    print(ingredient)


Ingredients:
Ingredients2 partsSauza®Silver Tequila1 partDeKuyper®Triple Sec Liqueur½ partDeKuyper®Blue Curacao Liqueur2 partsLime JuiceGarnish: Lime, Blueberries,SaltShopping listBuy Now


In [119]:
# Ingredients 뽑는 중

import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up the Chrome driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# URL of the cocktail detail page
url = 'https://www.thecocktailproject.com/drink-recipes/blue-margarita/'

# Open the cocktail detail page
driver.get(url)

# Wait for the ingredients to load
wait = WebDriverWait(driver, 10)
ingredients_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.ingredients-block')))

# Extract the ingredients
ingredients_text = ingredients_element.text

# Use regular expressions to split the text into separate lines
ingredients_list = re.findall(r'\d+\s*(?:½|¼)?\s*(?:parts|Garnish:)?\s*\S+\s*\S+\s*[\S\s]*?(?=\d+\s*(?:½|¼)?\s*(?:parts|Garnish:)?\s*\S+\s*\S+|$)', ingredients_text)

# Replace '½' with '1/2' and '\n' with ' ' for each element in the ingredients_list
ingredients_list = [ingredient.replace('½', '1/2').replace('\n', ' ').replace('¼', '1/4') for ingredient in ingredients_list]

ingredients_list
print("Ingredients:")
for ingredient in ingredients_list:
    print(ingredient)


Ingredients:
2 parts Sauza® Silver Tequila 
1 part DeKuyper® Triple Sec Liqueur 1/2 part DeKuyper® Blue Curacao Liqueur 
2 parts Lime Juice Garnish: Lime, Blueberries, Salt SHOPPING LIST BUY NOW


In [126]:
ingredients_list = [ingredient.replace('½', '\n1/2') for ingredient in ingredients_list]
ingredients_list = [ingredient.replace('\n', ' ').replace('¼', '1/4') for ingredient in ingredients_list]

ingredients_list

['2 parts Sauza® Silver Tequila ',
 '1 part DeKuyper® Triple Sec Liqueur 1/2 part DeKuyper® Blue Curacao Liqueur ',
 '2 parts Lime Juice Garnish: Lime, Blueberries, Salt SHOPPING LIST BUY NOW']